# Advanced Regression Techniques

- Importing required libraries

In [276]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [277]:
train_df = pd.read_csv('train.csv')

- We filtered the data according to 4 basic criterion: Relevance, Missing data, balance of the data and repeated columns


In [278]:
insufficient_data = ['Alley', 'Fence', 'MiscFeature', 'PoolQC', 'FireplaceQu', 
                     'LotFrontage']

imbalanced_data = ['3SsnPorch','BsmtCond', 'GarageQual', 'GarageCond', 'Heating', 'LandContour', 
                   'LandSlope', 'LowQualFinSF','PoolArea', 'RoofMatl', 'Street', 'Utilities']

nonsense_data = ['Condition1', 'Condition2', 'Functional', 'GarageType', 'GarageYrBlt', 
                 'BsmtHalfBath','LotConfig', 'LotShape', 'PavedDrive', 'MoSold', 'SaleCondition',
                 'YrSold','Id']

extra_data = ['GarageCars', 'GarageFinish', 'BsmtFullBath', 'YearBuilt']

train_df.drop(insufficient_data + imbalanced_data + nonsense_data + extra_data, axis = 1,
             inplace = True)

In [279]:
train_df.shape

(1460, 46)

- second_layer_checks to see if we missed anything with our logic

In [280]:
def correlation(dataset, threshold):
    corr_cols = []
    corr_matrix = dataset.corr(method = 'pearson')
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold):
                corr_cols.append((corr_matrix.columns[i],corr_matrix.columns[j], corr_matrix.iloc[i,j]))
    return corr_cols
#     print(dataset)

correlation(train_df,0.75)

[('1stFlrSF', 'TotalBsmtSF', 0.8195299750050355),
 ('TotRmsAbvGrd', 'GrLivArea', 0.8254893743088377),
 ('SalePrice', 'OverallQual', 0.7909816005838047)]

In [281]:
second_check = ['TotRmsAbvGrd', 'TotalBsmtSF']
train_df.drop(second_check, axis =1, inplace = True)
# train_df.shape

# Dealing with missing data

In [282]:
missing_data = train_df.isna().sum() / train_df.shape[0]
missing_data[missing_data > 0]

MasVnrType      0.005479
MasVnrArea      0.005479
BsmtQual        0.025342
BsmtExposure    0.026027
BsmtFinType1    0.025342
BsmtFinType2    0.026027
Electrical      0.000685
dtype: float64

In [283]:
train_df[(train_df['BsmtFinType1'].isna() == False) & (train_df['BsmtFinType2'].isna() == True)][['BsmtFinType1', 'BsmtFinType2']]

,BsmtFinType1,BsmtFinType2
332,GLQ,NaN


In [284]:
print(train_df['Electrical'].value_counts())
print()
print(train_df['MasVnrType'].value_counts())

SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64

None       864
BrkFace    445
Stone      128
BrkCmn      15
Name: MasVnrType, dtype: int64


- We should not remove the missing data with no basement. Likely to not be missing at random. We introduce new category called 'No Basement'
- For electrical, because we only have 1 missing row, we think it is reasonable to use the mode of the variable to fill this missing value
- For MasVnrType and MasVnrArea, we filled NA to assume that there is no MasVnr. Easiest way of dealing with inputting categorical for now

In [285]:
train_df.update(train_df[['BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']].fillna('No Basement'))
train_df['Electrical'] = train_df['Electrical'].fillna('SBrkr')
train_df['MasVnrType'] = train_df['MasVnrType'].fillna('None')
train_df['MasVnrArea'] = train_df['MasVnrArea'].fillna(0)

- Methods to input categorical: 1. Use the mode (introduce bias), 2. Missing values as new category, 3. Use classification algorithm to predict like Logistic regression / KNN

# Running Linear Regression model using sklearn

In [286]:
train_df.corr(method='pearson').style.background_gradient(cmap='coolwarm')

,MSSubClass,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,1stFlrSF,2ndFlrSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,Fireplaces,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,ScreenPorch,MiscVal,SalePrice
MSSubClass,1,-0.139781,0.0326277,-0.0593158,0.040581,0.023573,-0.0698357,-0.0656486,-0.140759,-0.251758,0.307886,0.0748532,0.131608,0.177354,-0.023438,0.281721,-0.0455693,-0.0986715,-0.0125794,-0.00610012,-0.0120366,-0.0260302,-0.00768329,-0.0842841
LotArea,-0.139781,1,0.105806,-0.00563627,0.0137884,0.103321,0.214103,0.11117,-0.00261836,0.299475,0.0509859,0.263116,0.126031,0.0142595,0.11969,-0.0177839,0.271364,0.180403,0.171698,0.0847738,-0.0183397,0.0431604,0.0380677,0.263843
OverallQual,0.0326277,0.105806,1,-0.0919323,0.550684,0.407252,0.239666,-0.0591187,0.308159,0.476224,0.295493,0.593007,0.5506,0.273458,0.101676,-0.183882,0.396765,0.562022,0.238923,0.308819,-0.113937,0.0648864,-0.0314062,0.790982
OverallCond,-0.0593158,-0.00563627,-0.0919323,1,0.0737415,-0.125694,-0.0462309,0.0402292,-0.136841,-0.144203,0.0289421,-0.0796859,-0.194149,-0.0607693,0.0129801,-0.0870009,-0.02382,-0.151521,-0.0033337,-0.0325888,0.0703562,0.0548105,0.0687768,-0.0778559
YearRemodAdd,0.040581,0.0137884,0.550684,0.0737415,1,0.176529,0.128451,-0.0677585,0.181133,0.240379,0.140024,0.287389,0.439046,0.183331,-0.0405809,-0.149598,0.112581,0.3716,0.205726,0.226298,-0.193919,-0.03874,-0.0102862,0.507101
MasVnrArea,0.023573,0.103321,0.407252,-0.125694,0.176529,1,0.261256,-0.0713296,0.113862,0.33985,0.1738,0.388052,0.272999,0.199108,0.102775,-0.0384501,0.247015,0.370884,0.159991,0.122528,-0.109907,0.0622477,-0.0295122,0.472614
BsmtFinSF1,-0.0698357,0.214103,0.239666,-0.0462309,0.128451,0.261256,1,-0.0501174,-0.495251,0.445863,-0.137079,0.208171,0.0585431,0.00426242,-0.107355,-0.0810069,0.260011,0.29697,0.204306,0.111761,-0.102303,0.0620206,0.00357147,0.38642
BsmtFinSF2,-0.0656486,0.11117,-0.0591187,0.0402292,-0.0677585,-0.0713296,-0.0501174,1,-0.209294,0.0971174,-0.0992603,-0.00963989,-0.0764439,-0.0321478,-0.0157281,-0.0407512,0.0469207,-0.0182266,0.0678983,0.00309256,0.0365433,0.0888713,0.00493978,-0.0113781
BsmtUnfSF,-0.140759,-0.00261836,0.308159,-0.136841,0.181133,0.113862,-0.495251,-0.209294,1,0.317987,0.00446909,0.240257,0.288886,-0.0411175,0.166643,0.0300859,0.0515749,0.183303,-0.00531642,0.129005,-0.00253785,-0.0125793,-0.0238366,0.214479
1stFlrSF,-0.251758,0.299475,0.476224,-0.144203,0.240379,0.33985,0.445863,0.0971174,0.317987,1,-0.202646,0.566024,0.380637,-0.119916,0.127401,0.0681006,0.410531,0.489782,0.235459,0.211671,-0.0652917,0.0887581,-0.0210957,0.605852


In [173]:
# Takes too long to run
# sns.pairplot(train_df)

In [287]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score

In [288]:
train_df['MSSubClass'].value_counts()
train_df = train_df.astype({'MSSubClass': 'object'})
y = train_df['SalePrice']
train_df.drop('SalePrice', axis = 1)

,MSSubClass,MSZoning,LotArea,Neighborhood,BldgType,HouseStyle,OverallQual,OverallCond,YearRemodAdd,RoofStyle,...,KitchenAbvGr,KitchenQual,Fireplaces,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,ScreenPorch,MiscVal,SaleType
0,60,RL,8450,CollgCr,1Fam,2Story,7,5,2003,Gable,...,1,Gd,0,548,0,61,0,0,0,WD
1,20,RL,9600,Veenker,1Fam,1Story,6,8,1976,Gable,...,1,TA,1,460,298,0,0,0,0,WD
2,60,RL,11250,CollgCr,1Fam,2Story,7,5,2002,Gable,...,1,Gd,1,608,0,42,0,0,0,WD
3,70,RL,9550,Crawfor,1Fam,2Story,7,5,1970,Gable,...,1,Gd,1,642,0,35,272,0,0,WD
4,60,RL,14260,NoRidge,1Fam,2Story,8,5,2000,Gable,...,1,Gd,1,836,192,84,0,0,0,WD
5,50,RL,14115,Mitchel,1Fam,1.5Fin,5,5,1995,Gable,...,1,TA,0,480,40,30,0,0,700,WD
6,20,RL,10084,Somerst,1Fam,1Story,8,5,2005,Gable,...,1,Gd,1,636,255,57,0,0,0,WD
7,60,RL,10382,NWAmes,1Fam,2Story,7,6,1973,Gable,...,1,TA,2,484,235,204,228,0,350,WD
8,50,RM,6120,OldTown,1Fam,1.5Fin,7,5,1950,Gable,...,2,TA,2,468,90,0,205,0,0,WD
9,190,RL,7420,BrkSide,2fmCon,1.5Unf,5,6,1950,Gable,...,2,TA,2,205,0,4,0,0,0,WD


In [289]:
qual_variables = list(train_df.columns[train_df.dtypes == 'object'])
quant_variables = list(train_df.columns[train_df.dtypes != 'object'])
# train_df[quant_variables]

In [292]:
dataframe_dic = {}
for i,j in zip(test,quant_variables):
    dataframe_dic[j] = i

quant_variables = pd.DataFrame(dataframe_dic)
qual_variables =pd.get_dummies(train_df[qual_variables], prefix = qual_variables, columns =  qual_variables)

ValueError: Must pass DataFrame with boolean values only

In [293]:
train_df = pd.merge(quant_variables, qual_variables, left_index = True, right_index = True)

In [295]:
X= train_df
X

,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,1stFlrSF,2ndFlrSF,...,KitchenQual_TA,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD
0,-0.207142,-0.091886,0.073480,-0.096897,0.375148,0.360616,-0.043379,-0.013513,-0.440659,-0.310370,...,0,0,0,0,0,0,0,0,0,1
1,0.651479,-0.071836,0.651479,0.651479,1.374795,-0.795151,1.374795,0.651479,0.651479,-0.795151,...,1,0,0,0,0,0,0,0,0,1
2,-0.517200,2.179628,-0.517200,-0.517200,-0.517200,-0.517200,-0.517200,0.381743,-0.517200,0.381743,...,0,0,0,0,0,0,0,0,0,1
3,0.878668,-0.429577,0.830215,-0.720298,0.733308,0.491040,0.975575,-0.574938,-1.689368,-1.689368,...,0,0,0,0,0,0,0,0,0,1
4,0.514104,-0.570750,0.325915,-0.570750,1.366489,-0.570750,0.458754,0.757643,-0.570750,-0.570750,...,0,0,0,0,0,0,0,0,0,1
5,0.575425,1.171992,0.092907,-0.499274,0.463568,0.632450,2.029558,0.910994,-0.973018,0.893448,...,1,0,0,0,0,0,0,0,0,1
6,-0.288653,-0.288653,-0.288653,-0.288653,-0.288653,-0.288653,-0.288653,-0.090220,-0.288653,-0.288653,...,0,0,0,0,0,0,0,0,0,1
7,-0.944591,-0.641228,-0.301643,-0.061670,-0.174865,-1.139286,-0.566519,-0.795173,0.871057,-0.967230,...,1,0,0,0,0,0,0,0,0,1
8,-0.793434,0.257140,-0.627826,-0.521734,-0.045611,-0.948691,1.374993,-0.143941,-0.363889,-0.221569,...,1,0,0,0,0,0,0,0,0,1
9,1.161852,-0.795163,1.189351,0.937276,1.617877,0.501875,-0.795163,1.457466,0.928110,-0.795163,...,1,0,0,0,0,0,0,0,0,1


In [272]:
lr = LinearRegression()
lr.fit(X,y)
train_score = lr.score(X,y)

1.0


In [296]:
X.head()

,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,1stFlrSF,2ndFlrSF,...,KitchenQual_TA,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD
0,-0.207142,-0.091886,0.073480,-0.096897,0.375148,0.360616,-0.043379,-0.013513,-0.440659,-0.310370,...,0,0,0,0,0,0,0,0,0,1
1,0.651479,-0.071836,0.651479,0.651479,1.374795,-0.795151,1.374795,0.651479,0.651479,-0.795151,...,1,0,0,0,0,0,0,0,0,1
2,-0.517200,2.179628,-0.517200,-0.517200,-0.517200,-0.517200,-0.517200,0.381743,-0.517200,0.381743,...,0,0,0,0,0,0,0,0,0,1
3,0.878668,-0.429577,0.830215,-0.720298,0.733308,0.491040,0.975575,-0.574938,-1.689368,-1.689368,...,0,0,0,0,0,0,0,0,0,1
4,0.514104,-0.570750,0.325915,-0.570750,1.366489,-0.570750,0.458754,0.757643,-0.570750,-0.570750,...,0,0,0,0,0,0,0,0,0,1


In [138]:
rr = Ridge(alpha = 100)
rr.fit(X,y)
Ridge_train_score = rr.score(X,y)
print(Ridge_train_score)

0.8567834509261874


In [131]:
test_df = pd.read_csv('test.csv')
test_df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [132]:
test_df.drop(insufficient_data + imbalanced_data + nonsense_data + extra_data, axis = 1,
             inplace = True)
test_df.drop(second_check, axis =1, inplace = True)
test_df.update(test_df[['BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']].fillna('No Basement'))
test_df['Electrical'] = test_df['Electrical'].fillna('SBrkr')
test_df['MasVnrType'] = test_df['MasVnrType'].fillna('None')
test_df['MasVnrArea'] = test_df['MasVnrArea'].fillna(0)
test_df = pd.get_dummies(test_df, prefix = cat_variables, columns =  cat_variables)

In [133]:
lr.predict(test_df)

ValueError: could not convert string to float: 'RH'

In [ ]:
# missing_vals = train_df[train_df.columns[train_df.isnull().sum() >0]]
# missing_vals
# # list(train_df.columns)
# to_drop = list(train_df.columns[train_df.isna().sum() > 300])
# train_df.drop(to_drop, axis =1, inplace = True)
# train_df[['GarageCars','GarageArea']]

In [ ]:
# vars_to_use = ['MSSubClass','LotArea', 'LotFrontage', '']

# train_df[['LotFrontage','LotArea']].corr()